## Importing libraries

In [20]:
import pandas as pd
import numpy as np
from collections import Counter
import csv
import Levenshtein
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

## Function call

In [21]:
def query_MS(query_list, resourcedb, LEN_flexibility, similarity):
    # Given a vector of CDR3 sequence, a database, length difference and sequence identity 
    # to search for toxoid CDR3 sequences, the function returns a dataframe with query toxoid CDR3 sequence
    # and the related BCR clone consenus
    new = []
    for item in query_list:
        aa1= query_SS(item, resourcedb, LEN_flexibility, similarity) ## Call query_SS function
        new.append(aa1)
        appended_data = pd.concat(new)
        appended_data.reset_index(inplace = True)
    return appended_data

def query_SS(sequence, resourcedb, LEN_flexibility, similarity):   
    # This function performs uses Levenshtein distance to allow the change in length of serach space and uses 
    # sequence identity to identify similar BCR sequences in a dataset.
    length= len(sequence)
    added_length = LEN_flexibility
    lower_length = length-added_length
    upper_length = length+added_length
    bins=[lower_length,upper_length]
    ## Generates LD score for all BCR sequences and query sequence
    LD_score = resourcedb.groupby(pd.cut(resourcedb['CDR3_IMGT_length'], bins=bins))['AA_JUNCTION_consensus'].apply(lambda x : process.extract(sequence, list(x), limit=30, scorer=fuzz.token_sort_ratio)).reset_index()
    ## Generates a df for further filtering
    Seq_LD = pd.DataFrame.from_records(LD_score['AA_JUNCTION_consensus'])
    Seq_LD = Seq_LD.transpose()
    Seq_LD =Seq_LD.rename(columns={0: "LD_RES"})
    Seq_LD[['AA_JUNCTION_consensus','LD_Ratio']] = pd.DataFrame(Seq_LD.LD_RES.tolist(), index= Seq_LD.index)
    Seq_LD['Query_Seq'] = sequence
    ## Only returns the sequence with a given sequence identity
    Seq_LD = Seq_LD[Seq_LD.LD_Ratio > similarity]
    return Seq_LD


## Loading data

In [11]:
dataset1 = pd.read_csv("BCR_dataset", sep='\t', lineterminator='\n', names=None) 
## This dataset has several additional column that we kept for generating further figures in the article.
## One can choose to select only relevant columns.

### Computing Consensus and creating a dataframe for identifying the anti-toxoid Abs

In [12]:
### Calculates consensus per clone
# Cluster_nt is the clone identification of BCRs
# CDR3_IMGT_Length is the length of CDR3 seuqence of the BCRs
# AA_JUNCTION is the CDR3 amino-acid sequence for each BCR
a2_nt = dataset1.groupby(['Cluster_nt','CDR3_IMGT_length'])['AA_JUNCTION'].apply(lambda x : Levenshtein.median(x)).reset_index()

### Create a dataframe for the consensus of the BCR
BCR_Consensus = pd.DataFrame(a2_nt)
BCR_Consensus = BCR_Consensus.rename(columns={"AA_JUNCTION": "AA_JUNCTION_consensus"})

### Merge the consensus information for each BCR
df_LD1 = pd.merge(dataset1, BCR_Consensus, on = (['Cluster_nt','CDR3_IMGT_length']), how = 'left')


### Vector of Tetanus, Diphtheria and Pertussis toxoid specific clones

In [14]:
## This creates a vector of the CDR3 sequences known in the literatutre for TT, DT and PT.
TT_Abs =["CARNLQGHYAMDVW" , "CARKGMGHYFDFW" , "CGKSYDYIRENLDSW" , "CARGVVPAGIPFDFW" , "CARARNYGFPHFFDFW" , "CARGEDCVGGSCYSADW" , "CARDYFHSGSQYFFDYW" , "CAKAPIIGPKYYFYMDVW" , "CAKAPIIGPKHYFYMDVW" , "CAKDRVRVVQAATTLDFW" , "CARKPRFYYDTSAWFEFW" , "CARDSVTNLGENLNFFPYW" , "CARDTVTPLGENLNYFAHW" , "CASTRCPENYGMDVW" , "CARGEPIRATVFGQPIPRGAWFDPW"]
DT_Abs =["GNTLDY" , "GFYYFDL" , "GGSPPFY" , "DRVYGLDV" , "GGVVSAVDY" , "ISVGVIPRDR" , "RVGKNNRGMAV" , "DQRLSVTYFDH" , "EHYLYYYGMDV" , "SAPKKYTGTHY" , "TDCSDGRCFFEF" , "HSWRTFASEFGY" , "HGVATIWGWFDP" , "RVGKNNYRGMAV" , "HGLHSTYTYFDP" , "HTWRTDSSESGY" , "DTKWELYGTFDY" , "HGLHSTYTYFDP" , "KPIVNTFRRKLDY" , "RPPYGGNLNWFDP" , "DRVSSGWPHRFDH" , "DAHKLSPTRYFFDY" , "GKYRGDRPRYYLDN" , "GERHYSNLRDYFDF" , "GERGYSNLRDYFDF" , "DSHKVSPTRYYFDY" , "VGHSGWGSHHWFDP" , "GVYRVGARPGSFHM" , "DKARGELRDNYFDY" , "DPGTGYSIDWYGFVI" , "VRWSSGGFNYYSMOV" , "DLWAALGVNGYGMDV" , "DAIAGATKRAVYFDF" , "SGRHPLLLRRSFFFDS" , "SARHPLLLRRSFFFDS" , "LTGGSLWYRAYWYFDL" , "SAMQTLLLRRSFFFDS" , "HVLRYFEWSLSPSDFDI" , "DLPEVPGLQVPGNFMKD" , "DKRDNIWRNFPSNWFDA" , "SSLPTYSYASGTYYFDY" , "SSLQTYSYASGTYYFDY" , "MQGGGGRYYYDYNEMDV" , "GFTIFGDLNPIGIYSMDV" , "AWVANNNRGSGYTPSAFHY" , "DRTPHTIFGEFLRELKNGMDV" , "DRSRLDAAPGGSDHYYYNGMDV" , "DRLRPPLTAAGSDHYYYYGMDV"]
PT_Abs = ['CARDTSFAPSGYGFDIS','CARVTGSGNFYGSYHYYYMDVW','CARENSIVGANWFDPW','CAKDREVLRFFLLPYYFDS','CARDRRKVRGVWAGYSGMDVW']


### Searching Tetanus, Diphtheria and Pertussis toxoid specific clones

In [22]:
# Here in the vector of TT, DT and PT CDR3 sequences are searched against the the consensus of the BCR clones.
PTx_clones= query_MS(PT_Abs, BCR_Consensus, 3, 70)
DTx_clones= query_MS(DT_Abs, BCR_Consensus, 5, 60)
TTx_clones= query_MS(TT_Abs, BCR_Consensus, 3, 70)

In [28]:
TTx_clones

,index,LD_RES,AA_JUNCTION_consensus,LD_Ratio,Query_Seq
0,0,"(CARCLQGSYAMDVW, 86)",CARCLQGSYAMDVW,86,CARNLQGHYAMDVW
1,1,"(CARPDGNSLGNAMDVW, 73)",CARPDGNSLGNAMDVW,73,CARNLQGHYAMDVW
2,2,"(CARVLEGGYHYFGMDVW, 71)",CARVLEGGYHYFGMDVW,71,CARNLQGHYAMDVW
3,0,"(CARAKTGMGFHWFDPW, 76)",CARAKTGMGFHWFDPW,76,CARKGMGHYFDFW
4,1,"(CARGMDGLAFDDFW, 74)",CARGMDGLAFDDFW,74,CARKGMGHYFDFW
5,2,"(CAKGGHDYNPFDYW, 74)",CAKGGHDYNPFDYW,74,CARKGMGHYFDFW
6,3,"(CARGAGIHWYFDLW, 74)",CARGAGIHWYFDLW,74,CARKGMGHYFDFW
7,4,"(CARADKLGLGYFDLW, 71)",CARADKLGLGYFDLW,71,CARKGMGHYFDFW
8,5,"(CARGSSGLQYYFDFW, 71)",CARGSSGLQYYFDFW,71,CARKGMGHYFDFW
9,0,"(CARRVVPAFMGFDFW, 80)",CARRVVPAFMGFDFW,80,CARGVVPAGIPFDFW


In [23]:
TT_clone_consensus = TTx_clones['AA_JUNCTION_consensus']
DT_clone_consensus = DTx_clones['AA_JUNCTION_consensus']
PT_clone_consensus = PTx_clones['AA_JUNCTION_consensus']

In [29]:
TT_clone_consensus

0           CARCLQGSYAMDVW
1         CARPDGNSLGNAMDVW
2        CARVLEGGYHYFGMDVW
3         CARAKTGMGFHWFDPW
4           CARGMDGLAFDDFW
5           CAKGGHDYNPFDYW
6           CARGAGIHWYFDLW
7          CARADKLGLGYFDLW
8          CARGSSGLQYYFDFW
9          CARRVVPAFMGFDFW
10      CARGPAAGTIIPYYFDFW
11     CAREGDVVVMPGPNWFDPW
12    CAREQDCSGGSCYRGALDYW
13      CARDGPDCGGDCYSDEFW
14      CARGCGGDCYSARTLDNW
15      CARGDCGGGSCFDAFDIW
16     CARDRYCTGGSCYSSFDYW
17       CARDQEYCGGDCYADYW
18       CARDYFGSGSRYFFDYW
19       CARDYFDSGSYYIFDYW
20     CARDFYDSSDGSPRYFDYW
21       CARDEGSGLSFERFDYW
22       CARDLKGGYSGYDFDYW
23       CARRDYDNDGFYFFDYW
24      CARGMRFYYDGISAYEFW
Name: AA_JUNCTION_consensus, dtype: object

In [24]:
TT_clones_Members = df_LD1[df_LD1['AA_JUNCTION_consensus'].isin(TT_clone_consensus)]
DT_clones_Members = df_LD1[df_LD1['AA_JUNCTION_consensus'].isin(DT_clone_consensus)]
PT_clones_Members = df_LD1[df_LD1['AA_JUNCTION_consensus'].isin(PT_clone_consensus)]

In [30]:
TT_clones_Members

,Unnamed: 0,index,Barcode,SequenceID,VDOMAIN_Functionality,VGENE_allele,VREGION_score,VREGION_identity,VREGION_identity_nt,VREGION_INS_DEL_identity,...,CDR3_NT_LEN,NAME,NAME_GENE,Cluster_nt,Cluster,Tag_ID,Upd_Cl_ID,IGHC_gene,IGKL_gene,AA_JUNCTION_consensus
3,3,3,GATGAGGTCAAGGTAA,GATGAGGTCAAGGTAA-1_contig_2,productive,Homsap IGHV1-18*01 F,1233,92.01,265/288 nt,NaN,...,45,IGHV1_IGHJ4_45,IGHV1-18_IGHJ4_45,288,39,NaN,NaN,IGHG1,NaN,CARDYFDSGSYYIFDYW
25,25,25,GCAAACTCACAGGAGT,GCAAACTCACAGGAGT-1_contig_2,productive,Homsap IGHV1-18*01 F,1125,87.85,253/288 nt,NaN,...,45,IGHV1_IGHJ4_45,IGHV1-18_IGHJ4_45,291,39,HTO3,P3,IGHG1,NaN,CARDYFGSGSRYFFDYW
67,67,67,AGAGCTTTCTAACCGA,AGAGCTTTCTAACCGA-1_contig_1,productive,Homsap IGHV1-18*01 F,1188,90.28,260/288 nt,NaN,...,48,IGHV1_IGHJ4_48,IGHV1-18_IGHJ4_48,322,40,HTO3,P1,IGHG1,NaN,CARGMRFYYDGISAYEFW
97,97,97,TACGGATCACCTGGTG,TACGGATCACCTGGTG-1_contig_1,productive,Homsap IGHV1-18*01 F,1251,92.71,267/288 nt,NaN,...,45,IGHV1_IGHJ4_45,IGHV1-18_IGHJ4_45,288,39,NaN,NaN,IGHG1,NaN,CARDYFDSGSYYIFDYW
102,102,102,CAGCTGGTCGACGGAA,CAGCTGGTCGACGGAA-1_contig_2,productive,Homsap IGHV1-18*01 F,1179,89.93,259/288 nt,NaN,...,45,IGHV1_IGHJ4_45,IGHV1-18_IGHJ4_45,288,39,HTO3,P3,IGHG1,NaN,CARDYFDSGSYYIFDYW
115,115,115,TTTGCGCTCGCGTAGC,TTTGCGCTCGCGTAGC-1_contig_2,productive,Homsap IGHV1-18*01 F,1215,91.32,263/288 nt,NaN,...,45,IGHV1_IGHJ4_45,IGHV1-18_IGHJ4_45,288,39,HTO5,B3,IGHG1,NaN,CARDYFDSGSYYIFDYW
120,120,120,ACAGCCGGTTTGACTG,ACAGCCGGTTTGACTG-1_contig_2,productive,Homsap IGHV1-18*01 F,1287,94.10,271/288 nt,NaN,...,45,IGHV1_IGHJ4_45,IGHV1-18_IGHJ4_45,288,39,HTO4,P2,IGHG1,NaN,CARDYFDSGSYYIFDYW
127,127,127,ACGCAGCTCACAGGCC,ACGCAGCTCACAGGCC-1_contig_2,productive,Homsap IGHV1-18*01 F,1215,91.32,263/288 nt,NaN,...,45,IGHV1_IGHJ4_45,IGHV1-18_IGHJ4_45,288,39,HTO4,P2,IGHG1,NaN,CARDYFDSGSYYIFDYW
132,132,132,TAAGAGACAGCATGAG,TAAGAGACAGCATGAG-1_contig_1,productive,Homsap IGHV1-18*01 F,1107,87.15,251/288 nt,NaN,...,48,IGHV1_IGHJ4_48,IGHV1-18_IGHJ4_48,322,40,HTO2,P2,IGHG1,NaN,CARGMRFYYDGISAYEFW
141,141,141,GCAATCACAGTGGGAT,GCAATCACAGTGGGAT-1_contig_2,productive,Homsap IGHV1-18*01 F,1287,94.10,271/288 nt,NaN,...,45,IGHV1_IGHJ4_45,IGHV1-18_IGHJ4_45,288,39,HTO4,P3,IGHG1,NaN,CARDYFDSGSYYIFDYW
